In [16]:
import pandas as pd
import numpy as np
import yfinance as yf

pd.set_option("mode.copy_on_write", True)

In [4]:
hedge_plan = pd.read_csv("final_hedge_plan.csv")
hedge_plan.head(5)

,date,hedge_type,cp_flag,strike_price,delta,gamma,mid_price,spread,contracts_needed
0,2018-02-22,delta,P,1525.0,-0.138812,0.018333,0.880,0.02,43660.534108
1,2018-02-22,gamma,C,1685.0,0.327754,0.047452,1.605,0.07,18491.325996
2,2018-02-23,delta,P,1585.0,-0.158830,0.024259,0.855,0.03,37461.561628
3,2018-02-23,gamma,C,1720.0,0.269699,0.054145,0.995,0.07,22061.705210
4,2018-02-26,delta,P,1615.0,-0.162650,0.026726,0.810,0.02,36090.782329


In [13]:
qqq_prices = pd.read_csv("Data/dataset1.csv")
qqq_prices['Date'] = pd.to_datetime(qqq_prices['Date'])
qqq_prices = qqq_prices.rename(columns={"Date": "date"})
qqq_prices = qqq_prices[1:]
print(qqq_prices.head(5))
returns_df = qqq_prices[['date', 'Return']].dropna()
returns_df = returns_df.set_index('date')
print(returns_df.head(5))

        date    Return  Realized_Volatility    Volume                 VIX  \
1 2018-02-22 -0.000122             0.283917  37074400  18.719999313354492   
2 2018-02-23  0.020449             0.292775  50096900  16.489999771118164   
3 2018-02-26  0.013260             0.296289  39266700  15.800000190734863   
4 2018-02-27 -0.012383             0.294650  42209900   18.59000015258789   
5 2018-02-28 -0.006417             0.294986  42936300  19.850000381469727   

      RSI_14                 VXN             FedRate  
1  45.694807   20.90999984741211  1.6050000190734863  
2  54.847494   18.15999984741211  1.6050000190734863  
3  68.915601    18.1200008392334  1.6080000400543213  
4  59.842025  20.489999771118164  1.6349999904632568  
5  61.920940  22.010000228881836  1.6180000305175781  
              Return
date                
2018-02-22 -0.000122
2018-02-23  0.020449
2018-02-26  0.013260
2018-02-27 -0.012383
2018-02-28 -0.006417


In [ ]:
# Hedge cost = contracts_needed * mid_price * 100 (options multiplier)
hedge_plan['hedge_cost'] = hedge_plan['contracts_needed'] * hedge_plan['mid_price'] * 100

# Aggregate hedge cost by date
daily_hedge_cost = hedge_plan.groupby('date')['hedge_cost'].sum().reset_index()

# Step 3: Merge returns and hedge cost
daily_hedge_cost['date'] = pd.to_datetime(daily_hedge_cost['date'])
daily_hedge_cost = daily_hedge_cost.set_index('date')

merged = returns_df.merge(daily_hedge_cost, how='left', left_index=True, right_index=True)
merged['hedge_cost'] = merged['hedge_cost'].fillna(0)

# Step 4: Simulate portfolio values
starting_value = 10_000_000  # Starting portfolio value

merged['unhedged_value'] = starting_value * (1 + merged['Return']).cumprod()

merged['hedged_value'] = starting_value
for i in range(1, len(merged)):
    previous_value = merged.iloc[i-1]['hedged_value']
    daily_return = merged.iloc[i]['Return']
    daily_cost = merged.iloc[i]['hedge_cost']
    merged.iloc[i, merged.columns.get_loc('hedged_value')] = (previous_value - daily_cost) * (1 + daily_return)

# Step 5: Preview results
merged[['unhedged_value', 'hedged_value', 'hedge_cost']].head()


/var/folders/t7/h0x6yb2j33qbz7kzz12cv3tw0000gn/T/ipykernel_8180/3278832875.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4696000.085233321' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged.iloc[i, merged.columns.get_loc('hedged_value')] = (previous_value - daily_cost) * (1 + daily_return)


,unhedged_value,hedged_value,hedge_cost
date,,,
2018-02-22,9.998785e+06,1.000000e+07,6.809985e+06
2018-02-23,1.020325e+07,4.696000e+06,5.398103e+06
2018-02-26,1.033855e+07,-3.726263e+05,5.063750e+06
2018-02-27,1.021053e+07,-6.983452e+06,6.698384e+06
2018-02-28,1.014500e+07,-1.372240e+07,6.827575e+06


In [8]:
# Step 7: Evaluate performance metrics

# Calculate daily portfolio returns
merged['unhedged_daily_return'] = merged['unhedged_value'].pct_change()
merged['hedged_daily_return'] = merged['hedged_value'].pct_change()

# Define evaluation function
def evaluate_performance(series):
    mean_return = series.mean()
    std_return = series.std()
    sharpe = mean_return / std_return * np.sqrt(252)
    sortino = mean_return / series[series < 0].std() * np.sqrt(252)
    var_95 = np.percentile(series, 5)
    es_95 = series[series <= var_95].mean()
    return {
        'Annual Return': mean_return * 252,
        'Annual Volatility': std_return * np.sqrt(252),
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'VaR (95%)': var_95,
        'Expected Shortfall (95%)': es_95
    }

# Evaluate unhedged and hedged
unhedged_perf = evaluate_performance(merged['unhedged_daily_return'].dropna())
hedged_perf = evaluate_performance(merged['hedged_daily_return'].dropna())

# Step 8: Hedge cost and variance reduction
#total_hedge_cost = merged['hedge_cost_scaled'].sum()
variance_unhedged = merged['unhedged_daily_return'].var()
variance_hedged = merged['hedged_daily_return'].var()
variance_reduction = (variance_unhedged - variance_hedged) / variance_unhedged

# Display all results
print("\nUnhedged Portfolio Performance:")
print(unhedged_perf)

print("\nHedged Portfolio Performance:")
print(hedged_perf)

print(f"\nTotal Hedge Cost Paid: ${total_hedge_cost:,.2f}")
print(f"Variance Reduction: {variance_reduction:.2%}")
print(f"Risk Reduction per $1,000 spent on hedging: {variance_reduction * 1000 / total_hedge_cost:.6f}")



Unhedged Portfolio Performance:
{'Annual Return': 0.20168119583140437, 'Annual Volatility': 0.241607278222034, 'Sharpe Ratio': 0.834748014693754, 'Sortino Ratio': 1.0887134607415303, 'VaR (95%)': -0.024879640846392725, 'Expected Shortfall (95%)': -0.03571119262328324}

Hedged Portfolio Performance:
{'Annual Return': 3.5792625651073373, 'Annual Volatility': 6.818375146372997, 'Sharpe Ratio': 0.5249436248768607, 'Sortino Ratio': 4.8350401238931875, 'VaR (95%)': -0.02228444082030314, 'Expected Shortfall (95%)': -0.04987770205842554}


NameError: name 'total_hedge_cost' is not defined

In [17]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 31.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
# streamlit_dashboard.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ── Load data ────────────────────────────────────────
hedge_plan = pd.read_csv("final_hedge_plan.csv")
hedge_plan['date'] = pd.to_datetime(hedge_plan['date'])

returns = pd.read_csv("Data/dataset1.csv")
returns['Date'] = pd.to_datetime(returns['Date'])
returns = returns.rename(columns={"Date": "date"})
returns = returns.set_index('date')
returns = returns.dropna()

# Hedge cost per day
hedge_plan['hedge_cost'] = hedge_plan['contracts_needed'] * hedge_plan['mid_price'] * 100
daily_cost = hedge_plan.groupby('date')['hedge_cost'].sum()
cost = daily_cost.reindex(returns.index, fill_value=0.0)

# Portfolio simulation
start_value = 10_000_000
unhedged = start_value * (1 + returns['Return']).cumprod()

hedged = [start_value]
for r, c in zip(returns['Return'].iloc[1:], cost.iloc[1:]):
    hedged.append((hedged[-1] - c) * (1 + r))
hedged = pd.Series(hedged, index=returns.index)

# Volatility estimation (21-day rolling window)
rolling_vol = returns['Return'].rolling(window=21).std() * np.sqrt(252)

# Simple VaR estimation
rolling_var = returns['Return'].rolling(window=21).quantile(0.05)

# ── Streamlit dashboard ──────────────────────────────
st.title("Market Risk Dashboard: QQQ Portfolio Hedging")

st.subheader("📈 Portfolio Value")
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(unhedged, label='Unhedged')
ax.plot(hedged, label='Hedged')
ax.legend()
ax.set_ylabel("Portfolio Value ($)")
st.pyplot(fig)

st.subheader("📊 Risk Metrics")

col1, col2, col3 = st.columns(3)

col1.metric("Current Volatility", f"{rolling_vol.iloc[-1]:.2%}")
col2.metric("Current 5% VaR", f"{rolling_var.iloc[-1]:.2%}")
col3.metric("Total Hedge Cost", f"${cost.sum():,.0f}")

# Risk Alert
if rolling_vol.iloc[-1] > 0.20:
    st.error(f"🚨 Volatility Alert! Realized vol is {rolling_vol.iloc[-1]:.2%}")
else:
    st.success(f"✅ Volatility under control: {rolling_vol.iloc[-1]:.2%}")

st.caption("Data: QQQ Returns + QQQ Options (Delta/Gamma Hedging)")


2025-04-25 19:05:23.684 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 19:05:23.762 
  command:

    streamlit run /opt/miniconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-25 19:05:23.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 19:05:23.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 19:05:23.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 19:05:23.780 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 19:05:24.167 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 19:05:24.168 Thread 'MainThread': 

DeltaGenerator()

In [3]:
import pandas as pd
import numpy as np

# Load QQQ return data
returns_df = pd.read_csv("Data/dataset1.csv")
returns_df['Date'] = pd.to_datetime(returns_df['Date'])
returns_df = returns_df.rename(columns={"Date": "date"})
returns_df = returns_df.set_index('date')
returns_df = returns_df.dropna(subset=['Return'])

# Simulate QQQ portfolio (buy-and-hold, unhedged)
initial_value = 10_000_000
returns_df['qqq_value'] = initial_value * (1 + returns_df['Return']).cumprod()
returns_df['qqq_daily_return'] = returns_df['qqq_value'].pct_change()

# Calculate performance metrics
def performance_metrics(series):
    daily_ret = series.dropna()
    mean_daily = daily_ret.mean()
    std_daily = daily_ret.std()
    downside_std = daily_ret[daily_ret < 0].std()
    
    annual_return = mean_daily * 252
    annual_volatility = std_daily * np.sqrt(252)
    sharpe = annual_return / annual_volatility
    sortino = annual_return / (downside_std * np.sqrt(252))
    var_95 = np.percentile(daily_ret, 5)
    es_95 = daily_ret[daily_ret <= var_95].mean()
    
    return {
        'Annual Return': annual_return,
        'Annual Volatility': annual_volatility,
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'VaR (95%)': var_95,
        'Expected Shortfall (95%)': es_95
    }

qqq_metrics = performance_metrics(returns_df['qqq_daily_return'])
qqq_metrics


{'Annual Return': 0.20168119583140437,
 'Annual Volatility': 0.241607278222034,
 'Sharpe Ratio': 0.834748014693754,
 'Sortino Ratio': 1.08871346074153,
 'VaR (95%)': -0.024879640846392725,
 'Expected Shortfall (95%)': -0.03571119262328324}